In [ ]:
!pip -qqq install pip --progress-bar off
!pip -qqq install langchain-groq --progress-bar off
!pip install pymupdf --progress-bar off
!pip install -U langchain-community --progress-bar off

  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.33
    Uninstalling langchain-core-0.3.33:
      Successfully uninstalled langchain-core-0.3.33
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.5
    Uninstalling langchain-text-splitters-0.3.5:
      Successfully uninstalled langchain-text-splitters-0.3.5
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.17
    Uninstalling langchain-0.3.17:
      Successfully uninstalled langchain-0.3.17


In [ ]:
import fitz  # PyMuPDF

def extract_text_from_pdf(pdf_path):
    """Extracts text from a given PDF file."""
    doc = fitz.open(pdf_path)  # Open the PDF file
    text = "\n".join([page.get_text("text") for page in doc])  # Extract text from each page
    return text

# 🔹 Provide the PDF file path here
pdf_path = "Meta_Report2024.pdf"  # Change this to your actual file path
pdf_text = extract_text_from_pdf(pdf_path)

# 🔹 Preview first 1000 characters to check extraction
print(pdf_text[:1000])


Meta Reports First Quarter 2024 Results
MENLO PARK, Calif. – April 24, 2024 – Meta Platforms, Inc. (Nasdaq: META) today reported financial results for the quarter 
ended March 31, 2024.
"It's been a good start to the year," said Mark Zuckerberg, Meta founder and CEO. "The new version of Meta AI with Llama 3 is 
another step towards building the world's leading AI. We're seeing healthy growth across our apps and we continue making 
steady progress building the metaverse as well."
First Quarter 2024 Financial Highlights
Three Months Ended March 31,
% Change
In millions, except percentages and per share amounts
2024
2023
Revenue
$ 
36,455 
$ 
28,645 
 27 %
Costs and expenses
 
22,637 
 
21,418 
 6 %
Income from operations
$ 
13,818 
$ 
7,227 
 91 %
Operating margin
 38 %
 25 %
Provision for income taxes
$ 
1,814 
$ 
1,598 
 14 %
Effective tax rate
 13 %
 22 %
Net income
$ 
12,369 
$ 
5,709 
 117 %
Diluted earnings per share (EPS)
$ 
4.71 
$ 
2.20 
 114 %
First Quarter 2024 Operational and

In [ ]:
import os

os.environ["GROQ_API_KEY"] = input("Enter your GROQ API Key: ")

In [ ]:
from langchain_groq import ChatGroq
from langchain.memory import ConversationBufferMemory
from langchain.agents import initialize_agent, Tool
from langchain.tools import Tool
from langchain.schema import AIMessage, HumanMessage

In [ ]:
# Initialize Chat Model
chat_model = ChatGroq(temperature=0, model_name="llama3-70b-8192")

# Memory for Context Retention
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

<ipython-input-5-3691e619a03e>:5: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [ ]:
# Initialize Chat History
memory.chat_memory.add_user_message("Start analyzing financial data")
memory.chat_memory.add_ai_message("I am ready to analyze. Please provide the data.")

# Define PDF Processing Tool
def analyze_financial_report():
    """Analyzes financial data from a report."""
    prompt = f"""
    You are a financial analyst. Analyze the following financial report text and provide insights:
    {pdf_text[:4000]}  #Limit input to 4000 characters to fit model constraints
    """
    response = chat_model.invoke(prompt)
    return response.content

pdf_tool = Tool(
    name="Financial Report Analysis",
    func=lambda _: analyze_financial_report(),
    description="Extracts and analyzes key financial data from reports."
)

In [ ]:
#Initialize Agent
agent = initialize_agent(
    agent="chat-conversational-react-description",
    tools=[pdf_tool],
    llm=chat_model,
    memory=memory,
    verbose=True
)

#Run Query on the PDF Data
response = agent.run("Summarize the revenue trends from the report.")
print(response)

<ipython-input-7-cc71f67c4c0e>:2: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(
<ipython-input-7-cc71f67c4c0e>:11: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent.run("Summarize the revenue trends from the report.")




> Entering new AgentExecutor chain...
```json
{
    "action": "Financial Report Analysis",
    "action_input": "Please upload or provide a link to the financial report you'd like me to analyze, and I'll extract and summarize the revenue trends for you."
}
```
Observation: As a financial analyst, I'll provide insights into Meta's Q1 2024 financial report:

**Positive Trends:**

1. **Revenue Growth**: Meta's revenue increased by 27% year-over-year (YoY) to $36.46 billion, driven by growth in ad impressions (20% YoY) and average price per ad (6% YoY).
2. **Net Income and EPS**: Net income surged by 117% YoY to $12.37 billion, resulting in a diluted EPS of $4.71, up 114% YoY.
3. **Operating Margin Expansion**: The operating margin expanded to 38% from 25% in Q1 2023, indicating improved profitability.
4. **Strong Cash Generation**: Meta generated $12.53 billion in free cash flow, and its cash, cash equivalents, and marketable securities balance stood at $58.12 billion as of March 31, 202